In [23]:
from torch import nn 
import torch 
import gym 
from collections import deque
import itertools
import numpy as np
import random 

In [24]:
## Inicializar hyperparametros
# Factor de descuento
gamma = 0.99

# Cantidad de transiciones de las que se hará un muestreo
batch_size= 32

# Máxima cantidad de transiciones guardadas
buffer_size= 50000

# minima cantidad de trnsiciones para computar gradientes y entrenar
min_replay_size = 1000 

# Inicio del Epsilon
epsilon_start = 1
epsilon_end = 0.02
epsilon_decay=10000

# Target update frecuency
target_update_frecuency = 1000


In [25]:
# Definir una clase con la red neuronal

class Network(nn.Module):
    def __init__(self, env):
        super().__init__()
        in_features = int(np.prod(env.observation_space.shape))
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.Tanh(),
            nn.Linear(64, env.action_space.n))
    
    def forward(self, x):
        return self.net(x)
    def act(self, obs):
          obs_t = torch.as_tensor(obs, dtype = torch.float32)
          q_values = self(obs_t.unsqueeze(0))

          max_q_index = torch.argmax(q_values, dim=1)[0]
          action = max_q_index.detach().item()

          return action


          pass
        

In [26]:
# Creación del ambiente
env = gym.make('CartPole-v0')
replay_buffer = deque(maxlen=buffer_size)
rew_buffer = deque([0.0], maxlen=100)
episode_reward = 0.0
online_net = Network(env)
target_net = Network(env)

target_net.load_state_dict(online_net.state_dict())
optimizer = torch.optim.Adam(online_net.parameters(), lr= 5e-4)

In [28]:
# Initialize Replay Buffer
obs = env.reset()
for _ in range (min_replay_size):
    action = env.action_space.sample()

    new_obs, rew, done, info,_ = env.step(action)
    transition = (obs, action, rew, done, new_obs)
    replay_buffer.apppend(transition)
    obs = new_obs
    if done:
        obs = env.reset()

AttributeError: 'collections.deque' object has no attribute 'apppend'

In [ ]:
# Main Training Loop

obs = env.reset
for step in itertools.count:
    epsilon = np.interp(step, [0, epsilon_decay], [epsilon_start, epsilon_end])
    rnd_sample = random.random
    if rnd_sample <= epsilon:
        action = env.action_space.sample()
    else:
        action = online_net.act(obs)

    new_obs, rew, done, _ = env.step(action) 
    transition = (obs, action, rew, done, new_obs)
    replay_buffer.append(transition)
    obs = new_obs
    episode_reward += rew

    if done:
        obs = env.reset()
        rew_buffer.append(episode_reward)
        episode_reward = 0.0

TypeError: 'type' object is not iterable

In [ ]:
# Start Gradient Step
transition = random.sample(replay_buffer, batch_size)
obses = np.asarray([t[0] for t in transition])
action = np.asarray([t[1] for t in transition])
rews = np.asarray([t[2] for t in transition])
dones = np.asarray([t[3] for t in transition])
new_obses = np.asarray([t[4] for t in transition])


obses_t = torch.as_tensor(obses, dtype = torch.float32).unsqueeze(-1)
actions_t = torch.as_tensor(action, dtype = torch.float32).unsqueeze(-1)
rews_t = torch.as_tensor(rews, dtype = torch.float32).unsqueeze(-1)
dones_t = torch.as_tensor(dones, dtype = torch.float32).unsqueeze(-1)
new_obses_t = torch.as_tensor(new_obs, dtype = torch.float32)



In [ ]:
# Compute targets
target_q_values = target_net(new_obses_t)
max_target_q_values = target_q_values.max(dim = 1, keepdim = True)[0]
target = rews_t + gamma

In [ ]:
# Compute loss
q_values = online_net(obses_t)
action_q_values = torch.gather(input=q_values, dim = 1, index=actions_t)
loss = nn.functional.smooth_l1_loss(action_q_values, target)

In [ ]:
#Gradient Descent
optimizer.zero_grad
loss.backward()
optimizer.step()

# Update Target Network
if step % target_update_frecuency == 0:
    target_net.load_state_dict(online_net.state_dict())

# Logging 
if step % 1000 == 0:
    print()
    print('Paso', step)
    print('Avg rew', np.mean(rew_buffer))